In [1]:
import skimage
import tensorflow as tf
from tensorflow import keras
import numpy as np
from pathlib import Path
import os
import random

In [2]:
## Set seed for reproducibility
SEED = 42

# `PYTHONHASHSEED` environment variable
os.environ['PYTHONHASHSEED'] = str(SEED)

# Python built-in random, numpy(+ scikit) and tensorflow seed
tf.keras.utils.set_random_seed(SEED)

In [3]:
DATASET_PATH = "C:/Users/Alessandro/Desktop/università/visual image/dataset/"
target_shape = (200,200)
train_path = DATASET_PATH + "train"
x_train = list(Path(train_path).rglob("*.jpg"))
y_train = [path.parts[-2] for path in x_train]
test_path = DATASET_PATH + "test"
x_test = list(Path(test_path).rglob("*.jpg"))
y_test = [path.parts[-2] for path in x_test]

In [4]:
embedding = keras.models.load_model("./final_siamese_embedding_model/")

In [5]:
from skimage.io import imread
from tqdm import tqdm

embedding_train = []
for image_path in tqdm(x_train):
    img = imread(image_path)
    img = skimage.transform.resize(img, target_shape)
    img = np.dstack([img])
    embedding_train.extend([embedding(
        tf.expand_dims(img,
            axis=0
            )).numpy()[0]])

embedding_test = []
for image_path in tqdm(x_test):
    img = imread(image_path)
    img = skimage.transform.resize(img, target_shape)
    img = np.dstack([img])
    embedding_test.extend([embedding(
        tf.expand_dims(img,
            axis=0
            )).numpy()[0]])

100%|██████████| 600/600 [01:28<00:00,  6.80it/s]


In [6]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

classifier = RandomForestClassifier(random_state=42)
classifier.fit(embedding_train, y_train)
print("score: ", classifier.score(embedding_test, y_test))
predicted = classifier.predict(embedding_test)
a = classification_report(y_test, predicted)
print(a)
print(confusion_matrix(y_test, predicted))

score:  0.8533333333333334
              precision    recall  f1-score   support

      savory       0.83      0.88      0.86       300
    unsavory       0.88      0.82      0.85       300

    accuracy                           0.85       600
   macro avg       0.85      0.85      0.85       600
weighted avg       0.85      0.85      0.85       600

[[265  35]
 [ 53 247]]


In [10]:
classifier_svm = svm.NuSVC(nu=0.3, random_state=42)
classifier_svm.fit(embedding_train, y_train)
print("score: ", classifier_svm.score(embedding_test, y_test))
predicted = classifier_svm.predict(embedding_test)
a = classification_report(y_test, predicted)
print(a)
print(confusion_matrix(y_test, predicted))

score:  0.895
              precision    recall  f1-score   support

      savory       0.88      0.91      0.90       300
    unsavory       0.91      0.88      0.89       300

    accuracy                           0.90       600
   macro avg       0.90      0.90      0.89       600
weighted avg       0.90      0.90      0.89       600

[[274  26]
 [ 37 263]]


In [8]:
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(embedding_train, y_train)
print("score: ", classifier.score(embedding_test, y_test))
predicted = classifier.predict(embedding_test)
a = classification_report(y_test, predicted)
print(a)
print(confusion_matrix(y_test, predicted))

score:  0.7816666666666666
              precision    recall  f1-score   support

      savory       0.79      0.77      0.78       300
    unsavory       0.78      0.79      0.78       300

    accuracy                           0.78       600
   macro avg       0.78      0.78      0.78       600
weighted avg       0.78      0.78      0.78       600

[[232  68]
 [ 63 237]]


c:\Users\Alessandro\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
c:\Users\Alessandro\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eli

In [19]:
import pickle
with open("./Classifier/svm_final_nu03.sav", "wb") as handle:
    pickle.dump(classifier_svm, handle, pickle.HIGHEST_PROTOCOL)

In [20]:
with open("./Features/feature_final.pkl", "wb") as handle:
    pickle.dump(embedding_train, handle, pickle.HIGHEST_PROTOCOL)

with open("./Features/path_final.pkl", "wb") as handle:
    pickle.dump(x_train, handle, pickle.HIGHEST_PROTOCOL)